In [76]:
# lod: loading imports
import pandas as pd
import numpy as np
from pandas import json_normalize
from matplotlib import pyplot as plt
import json
import codecs
import typing

In [77]:
# lod: https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe
# loading the cities.json file into a pandas dataframe (like a table)
df = None
with codecs.open("cities.json", encoding="utf-8", mode="r") as read_file:
    data = json.load(read_file)
    df = json_normalize(data)

drops = ["id", "state_id", "state_code", "state_name", "country_id", "country_name"]
for drop in drops:
    df.drop(drop, axis=1, inplace=True)

# merge name and country_name into one column
df["city"] = df["name"] + "-" + df["country_code"]
df.drop("name", axis=1, inplace=True)
df.drop("country_code", axis=1, inplace=True)
# move city column to the front
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [78]:
cities = [
    "Lubumbashi-CD",
    "Casablanca-MA",
    "Johannesburg-ZA",
    "Nairobi-KE",
    "Cairo-EG",
    "Lagos-NG",
]

use_df = df[df["city"].isin(cities)]

In [79]:
from math import radians, cos, sin, sqrt

# lod: https://stackoverflow.com/questions/20654918/python-how-to-speed-up-calculation-of-distances-between-cities

# Constants defined by the World Geodetic System 1984 (WGS84)
A = 6378.137
B = 6356.7523142
ESQ = 6.69437999014 * 0.001


# lod: trying to understand this, it basically makes a point in earth centered earth fixed coordinates
# lod: https://en.wikipedia.org/wiki/ECEF
def geodetic2ecef(lat, lon, alt=0):
    """Convert geodetic coordinates to ECEF."""
    lat, lon = radians(lat), radians(lon)
    xi = sqrt(1 - ESQ * sin(lat))
    x = (A / xi + alt) * cos(lat) * cos(lon)
    y = (A / xi + alt) * cos(lat) * sin(lon)
    z = (A / xi * (1 - ESQ) + alt) * sin(lat)
    return x, y, z


def euclidean_distance(distance):
    """Return the approximate Euclidean distance corresponding to the
    given great circle distance (in km).

    """
    return 2 * A * sin(distance / (2 * B))


ecef_all_cities = [
    geodetic2ecef(float(lat), float(lon))
    for lat, lon in zip(df["latitude"], df["longitude"])
]
min_all = min(ecef_all_cities)
max_all = max(ecef_all_cities)

ecef_cities = [
    geodetic2ecef(float(lat), float(lon))
    for lat, lon in zip(use_df["latitude"], use_df["longitude"])
]

def distance_to(start: str, end: str) -> float:
    """Return the great circle distance between two cities."""
    start_idx = use_df.index[use_df["city"] == start].tolist()[0]
    end_idx = use_df.index[use_df["city"] == end].tolist()[0]
    start_ecef = ecef_all_cities[start_idx]
    end_ecef = ecef_all_cities[end_idx]
    return euclidean_distance(np.linalg.norm(np.array(start_ecef) - np.array(end_ecef)))


In [80]:
# lod: we make a df with a mapping from city to city containing the distance between them
distances = pd.DataFrame(
    [
        [distance_to(start, end) for end in cities]
        for start in cities
    ],
    columns=cities,
    index=cities,
)

In [81]:
import itertools

# lod: https://stackoverflow.com/questions/104420/how-do-i-generate-all-permutations-of-a-list
routes_uncategorised = list(itertools.permutations(cities))
routes = dict()

# lod: this works by just getting all the permutations then going through from the first to the last
# city and adding the distance between them 
for route in routes_uncategorised:
    real_route = route[1:]
    distance = 0
    first = route[0]
    previous = route[0]
    if first not in routes:
        routes[first] = list()
    for index, location in enumerate(real_route):
        distance += distances[previous][location]
        previous = location
    routes[first].append((route, distance))

# lod: easy enough

In [82]:
# lod: https://en.wikipedia.org/wiki/Simulated_annealing
# I have no idea what this is but I'm going to research and read about it, 
# apparently you can use it to solve the travelling salesman problem which we are kind of doing

# lod: what I gathered from https://bminaiev.github.io/simulated-annealing
# there's an objective function that we want to minimize, in this case the distance
# then we start with an initial solution
# over many iterations we tweak the solution slightly and if the new solution is better we keep it
# then after a certain number of iterations we stop and return the best solution we found

class Solution:
    # dict<K1, V1>
    # K1: city name
    # V1: list of distances for each route
    # V1 = [distance1, distance2, distance3, ...]
    # where each distance maps to the codeblock above's 
    calculated_distances: typing.Dict[str, typing.List[float]]
    # for each city, store the current optimal route 
    paths: typing.Dict[str, int]

    def __init__(self, calculated_distances: typing.Dict[str, typing.List[float]]):
        self.calculated_distances = calculated_distances
        self.paths = dict()
        for city, distance in calculated_distances.items():
            self.paths[city] = 0

    def changed_paths(self, f: typing.Callable[[typing.Any], typing.Dict[str, int]]):
        new_solution = Solution(self.calculated_distances)
        new_solution.paths = f(self)
        return new_solution

# we already have gathered the score for each route so we can just use that as the objective function
def objective(solution: Solution) -> float:
    # we want to minimize the difference in distance between the cities
    # I guess we can use standard deviation for this
    distances = []
    for city, route in solution.paths.items():
        distances.append(solution.calculated_distances[city][route])
    return np.std(distances)

def tweak_paths(solution: Solution) -> typing.Dict[str, int]:
    paths = solution.paths
    new_paths = paths.copy()
    # randomly change one of the paths
    city = np.random.choice(list(paths.keys()))
    new_paths[city] = np.random.randint(0, len(solution.calculated_distances[city]))
    return new_paths

def tweak(solution: Solution) -> Solution:
    # randomly change one of the paths
    new_solution = solution.changed_paths(tweak_paths)
    old_score = objective(solution)
    new_score = objective(new_solution)
    if new_score < old_score:
        return new_solution
    else:
        return solution

# actual solution solving here
def anneal(solution: Solution, iterations: int) -> Solution:
    for i in range(iterations):
        solution = tweak(solution)
    return solution

In [ ]:
# lod: I'm going to try to run this and see what happens
def extract_distance_dict(routes: dict) -> typing.Dict[str, typing.List[float]]:
    distances = dict()
    for city, routes in routes.items():
        distances[city] = [route[1] for route in routes]
    return distances

initial_solution = Solution(extract_distance_dict(routes))
final_solution = anneal(initial_solution, 10000)
print(final_solution.paths)
print(objective(final_solution))
for city, route in final_solution.paths.items():
    print(routes[city][route])

# TODO(lod): make sure that at least 2 teams are together at each point of the route
# this is probably going to be very hard :(

{'Lubumbashi-CD': 91, 'Casablanca-MA': 47, 'Johannesburg-ZA': 59, 'Nairobi-KE': 38, 'Cairo-EG': 35, 'Lagos-NG': 10}
29.92937637533317
(('Lubumbashi-CD', 'Cairo-EG', 'Lagos-NG', 'Casablanca-MA', 'Nairobi-KE', 'Johannesburg-ZA'), np.float64(19919.06855237856))
(('Casablanca-MA', 'Johannesburg-ZA', 'Lagos-NG', 'Cairo-EG', 'Nairobi-KE', 'Lubumbashi-CD'), np.float64(19922.93573964407))
(('Johannesburg-ZA', 'Nairobi-KE', 'Casablanca-MA', 'Lagos-NG', 'Cairo-EG', 'Lubumbashi-CD'), np.float64(19919.068552378558))
(('Nairobi-KE', 'Casablanca-MA', 'Cairo-EG', 'Johannesburg-ZA', 'Lubumbashi-CD', 'Lagos-NG'), np.float64(19882.93288571129))
(('Cairo-EG', 'Casablanca-MA', 'Johannesburg-ZA', 'Lagos-NG', 'Nairobi-KE', 'Lubumbashi-CD'), np.float64(19971.875943891697))
(('Lagos-NG', 'Lubumbashi-CD', 'Johannesburg-ZA', 'Cairo-EG', 'Casablanca-MA', 'Nairobi-KE'), np.float64(19882.932885711292))


In [ ]:
# lod: using plotly to plot the cities in 3d
# lod: https://stackoverflow.com/questions/38364435/make-3d-plot-interactive-in-jupyter-notebook
import plotly
import plotly.graph_objs as go
from plotly.graph_objs.layout import Scene
plotly.offline.init_notebook_mode()
trace = go.Scatter3d(
    x=[x for x, _, _ in ecef_cities],
    y=[y for _, y, _ in ecef_cities],
    z=[z for _, _, z in ecef_cities],
    mode="markers",
    marker={
        "size": 5,
    },
    text=use_df["city"],
)

layout = go.Layout(
    title="Cities in ECEF",
    margin={"l": 0, "r": 0, "b": 0, "t": 0},
    scene=Scene(
        {
            "xaxis": {
                "range": [-8000, 8000],
            },
            "yaxis": {
                "range": [-8000, 8000],
            },
            "zaxis": {
                "range": [-8000, 8000],
            },
        }
    ),
)
data = [trace]
plot_figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(plot_figure)

[(5538.004288450576, 2880.3698794848005, -1279.6461728198524),
 (5276.822581202903, -700.1278042892781, 3511.418429547304),
 (5043.3709378518115, 2686.536109490775, -2793.2336178715655)]